In [9]:
import pandas as pd
import numpy as np
import glob
import os
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils import class_weight
import pickle
import random

Category = "Country"
Africa = ['DZ', 'EG', 'MA', 'ZA', 'TN']
Asia = ['BH', 'HK', 'IN', 'ID', 'IL', 'JP', 'JO', 'KW', 
        'LB', 'MY', 'OM', 'PS', 'PH', 'QA', 'SA', 'SG', 
        'TW', 'TH', 'AE', 'VN']
Europe = ['AD', 'AT', 'BE', 'BG', 'CY', 'CZ', 'DK', 'EE', 
          'FI', 'FR', 'DE', 'GR', 'HU', 'IS', 'IE', 'IT', 
          'LV', 'LI', 'LT', 'LU', 'MT', 'MC', 'NL', 'NO', 
          'PL', 'PT', 'RO', 'SK', 'ES', 'SE', 'CH', 'TR', 'GB']
NorthAmerica = ['CA', 'CR', 'DO', 'SV', 'GT', 'HN', 'MX', 'NI', 'PA', 'US']
SouthAmerica = ['AR', 'BO', 'BR', 'CL', 'CO', 'EC', 'PY', 'PE', 'UY']
Oceania = ['AU', 'NZ']

countriesOfInterest = Africa + Asia + Europe + NorthAmerica + SouthAmerica + Oceania
#countriesOfInterest = ["HK", "JP", 'ZA', 'TN', 'TR', 'GB', 'MX', 'US', 'CO', 'EC', 'AU', 'NZ']

w_length = 200

### Consolidate and pickle country files

In [2]:
for country in countriesOfInterest:
    if len(glob.glob("Raw Track Data\\" + country + ".p")) == 0 :
        hold = pd.DataFrame()
    else:
        hold = pickle.load( open( "Raw Track Data\\" + country + ".p", "rb" ) )
    print(country, hold.shape)
    countryTracks = glob.glob("Raw Track Data\\" + country + "*.csv")
    print(country, len(countryTracks))
    for file in countryTracks:
        new = pd.read_csv(file)
        new["Country"] = file[15:17]
        new["Year"] = file[18:22]
        new["Playlist"] = file[23:-4]
        hold = hold.append(new)
        pickle.dump( hold, open( "Raw Track Data\\" + country + ".p", "wb" ) )
        os.remove(file)
    hold = hold.drop_duplicates(["track_id", "start"])
    pickle.dump( hold, open( "Raw Track Data\\" + country + ".p", "wb" ) )

DZ (18257, 34)
DZ 79
EG (392985, 34)
EG 85
MA (0, 34)
MA 72
ZA (1766791, 34)
ZA 63
TN (127000, 34)
TN 63
BH (20133, 34)
BH 66
HK (749948, 34)
HK 49
IN (96116, 34)
IN 48
ID (1527456, 34)
ID 44
IL (1224241, 34)
IL 44
JP (1678485, 34)
JP 36
JO (31552, 34)
JO 55
KW (17765, 34)
KW 65
LB (17973, 34)
LB 55
MY (467310, 34)
MY 55
OM (2759, 34)
OM 64
PS (57312, 34)
PS 53
PH (812069, 34)
PH 47
QA (1643, 34)
QA 64
SA (0, 34)
SA 67
SG (570820, 34)
SG 51
TW (2087452, 34)
TW 56
TH (1373744, 34)
TH 54
AE (199542, 34)
AE 67
VN (1488943, 34)
VN 54
AD (16976, 34)
AD 36
AT (490675, 34)
AT 30
BE (714640, 34)
BE 26
BG (7218, 34)
BG 36
CY (88872, 34)
CY 44
CZ (72625, 34)
CZ 40
DK (2540912, 34)
DK 44
EE (1345, 34)
EE 48
FI (1992455, 34)
FI 47
FR (1655397, 34)
FR 40
DE (495993, 34)
DE 40
GR (0, 34)
GR 48
HU (0, 34)
HU 48
IS (222275, 34)
IS 44
IE (1362040, 34)
IE 40
IT (902453, 34)
IT 38
LV (1290, 34)
LV 48
LI (0, 34)
LI 48
LT (0, 34)
LT 48
LU (1402, 34)
LU 48
MT (0, 34)
MT 48
MC (4072, 34)
MC 48
NL (1321251, 3

In [3]:
hold

,start,duration,confidence,loudness_start,loudness_max_time,loudness_max,p1,p2,p3,p4,...,t7,t8,t9,t10,t11,t12,track_id,Country,Year,Playlist
681,0.00000,0.49914,0.000,-60.000,0.00000,-60.000,0.019,0.040,0.045,0.024,...,14.833,5.359,-27.228,0.973,-10.640,-7.228,spotify:track:7imVDfvNz2H4szraikF3Hd,NZ,2015,37i9dQZF1DX9TriA5Rm2k8
682,0.49914,0.22644,1.000,-60.000,0.03370,-10.164,0.646,0.420,0.522,0.560,...,-20.233,-14.167,-6.882,34.476,41.585,7.220,spotify:track:7imVDfvNz2H4szraikF3Hd,NZ,2015,37i9dQZF1DX9TriA5Rm2k8
683,0.72558,0.12181,0.381,-31.803,0.00782,-26.013,0.940,0.858,0.439,0.460,...,16.783,-18.108,13.722,32.195,-23.695,-3.539,spotify:track:7imVDfvNz2H4szraikF3Hd,NZ,2015,37i9dQZF1DX9TriA5Rm2k8
684,0.84739,0.60340,0.952,-26.892,0.18692,-14.485,0.073,0.149,0.097,0.237,...,64.849,-42.428,21.399,50.362,-43.406,-18.990,spotify:track:7imVDfvNz2H4szraikF3Hd,NZ,2015,37i9dQZF1DX9TriA5Rm2k8
685,1.45079,0.20980,0.775,-21.574,0.04262,-13.192,0.296,0.377,0.302,0.450,...,38.661,-2.748,52.698,13.987,-17.991,-5.808,spotify:track:7imVDfvNz2H4szraikF3Hd,NZ,2015,37i9dQZF1DX9TriA5Rm2k8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37930,170.81125,0.89347,0.460,-8.179,0.01010,0.215,0.414,0.409,0.446,0.267,...,-25.145,-43.411,-12.199,-13.408,-33.778,3.549,spotify:track:1Ut1A8UaNqGuwsHgWq75PW,NZ,2007,37i9dQZF1DXamBAn43YNB9
37931,171.70472,0.50540,0.060,-27.415,0.04906,-24.362,0.181,0.193,0.166,0.119,...,-34.940,-6.455,-4.831,-9.044,-17.405,5.136,spotify:track:1Ut1A8UaNqGuwsHgWq75PW,NZ,2007,37i9dQZF1DXamBAn43YNB9
37932,172.21011,0.25111,0.176,-31.961,0.00673,-29.397,0.119,0.289,0.247,0.100,...,-53.636,0.270,0.474,-1.909,-0.930,5.495,spotify:track:1Ut1A8UaNqGuwsHgWq75PW,NZ,2007,37i9dQZF1DXamBAn43YNB9
37933,172.46122,0.37188,0.097,-35.471,0.02143,-32.060,0.173,0.264,0.269,0.109,...,-44.294,-2.119,-13.087,7.312,0.573,0.544,spotify:track:1Ut1A8UaNqGuwsHgWq75PW,NZ,2007,37i9dQZF1DXamBAn43YNB9


### Remove non-unique tracks

In [4]:
allTracks = pd.DataFrame()
for country in countriesOfInterest:
    hold = pickle.load( open( "Raw Track Data\\" + country + ".p", "rb" ) )
    hold = hold[["Country", "Year", "track_id", "Playlist"]]
    allTracks = allTracks.append(hold)
allTracks

,Country,Year,track_id,Playlist
19557,DZ,2016,spotify:track:1yxafhHFJMYkA8oSlLz68Q,37i9dQZF1DWVSIz2AGspV4
19558,DZ,2016,spotify:track:1yxafhHFJMYkA8oSlLz68Q,37i9dQZF1DWVSIz2AGspV4
19559,DZ,2016,spotify:track:1yxafhHFJMYkA8oSlLz68Q,37i9dQZF1DWVSIz2AGspV4
19560,DZ,2016,spotify:track:1yxafhHFJMYkA8oSlLz68Q,37i9dQZF1DWVSIz2AGspV4
19561,DZ,2016,spotify:track:1yxafhHFJMYkA8oSlLz68Q,37i9dQZF1DWVSIz2AGspV4
...,...,...,...,...
37930,NZ,2007,spotify:track:1Ut1A8UaNqGuwsHgWq75PW,37i9dQZF1DXamBAn43YNB9
37931,NZ,2007,spotify:track:1Ut1A8UaNqGuwsHgWq75PW,37i9dQZF1DXamBAn43YNB9
37932,NZ,2007,spotify:track:1Ut1A8UaNqGuwsHgWq75PW,37i9dQZF1DXamBAn43YNB9
37933,NZ,2007,spotify:track:1Ut1A8UaNqGuwsHgWq75PW,37i9dQZF1DXamBAn43YNB9


In [5]:
hold = allTracks.groupby("track_id").nunique()
print(hold.shape[0], "Unique tracks per country")
keep = hold[hold.Country==1].index
allTracks = allTracks[allTracks.track_id.isin(keep)]
print(len(keep), "Unique tracks across countries")

88493 Unique tracks per country
62701 Unique tracks across countries


In [6]:
for country in countriesOfInterest:
    hold = pickle.load( open( "Raw Track Data\\" + country + ".p", "rb" ) )
    hold = hold[hold.track_id.isin(keep)]
    pickle.dump( hold, open( "Raw Track Data\\" + country + ".p", "wb" ) )

### Create training, testing, and validation datasets

In [7]:
for country in countriesOfInterest:
    hold = pickle.load( open( "Raw Track Data\\" + country + ".p", "rb" ) )
    UniqueTracks = pd.unique(hold.track_id)
    testTracks = np.random.choice(UniqueTracks, int(len(UniqueTracks) * .2), replace = False)
    test = hold.loc[hold.track_id.isin(testTracks)]
    hold = hold.loc[~hold.track_id.isin(testTracks)]
    UniqueTracks = pd.unique(hold.track_id)
    testTracks = np.random.choice(UniqueTracks, int(len(UniqueTracks) * .2), replace = False)
    val = hold.loc[hold.track_id.isin(testTracks)]
    train = hold.loc[~hold.track_id.isin(testTracks)]
    pickle.dump( train, open( "Raw Track Data\\" + country + "_train.p", "wb" ) )
    pickle.dump( test, open( "Raw Track Data\\" + country + "_test.p", "wb" ) )
    pickle.dump( val, open( "Raw Track Data\\" + country + "_val.p", "wb" ) )

In [8]:
countriesOfInterest = ["HK", "JP", 'ZA', 'TN', 'TR', 'GB', 'MX', 'US', 'CO', 'EC', 'AU', 'NZ']
for country in countriesOfInterest:
    train = pickle.load( open( "Raw Track Data\\" + country + "_train.p", "rb" ) )
    test = pickle.load( open( "Raw Track Data\\" + country + "_test.p", "rb" ) )
    val = pickle.load( open( "Raw Track Data\\" + country + "_val.p", "rb" ) )
    print(country, "train", len(pd.unique(train.track_id)), 
          "\t test",len(pd.unique(test.track_id)) , 
          "\t val", len(pd.unique(val.track_id)))

HK train 567 	 test 176 	 val 141
JP train 1556 	 test 486 	 val 389
ZA train 1404 	 test 438 	 val 350
TN train 36 	 test 11 	 val 8
TR train 1342 	 test 419 	 val 335
GB train 651 	 test 203 	 val 162
MX train 772 	 test 241 	 val 192
US train 1547 	 test 483 	 val 386
CO train 1192 	 test 372 	 val 297
EC train 352 	 test 109 	 val 87
AU train 692 	 test 216 	 val 173
NZ train 434 	 test 135 	 val 108


In [16]:
train_arr = []
test_arr = []
val_arr = []
for country in countriesOfInterest:
    train = pickle.load( open( "Raw Track Data\\" + country + "_train.p", "rb" ) )
    test = pickle.load( open( "Raw Track Data\\" + country + "_test.p", "rb" ) )
    val = pickle.load( open( "Raw Track Data\\" + country + "_val.p", "rb" ) )
    print(country, "train", len(pd.unique(train.track_id)), 
          "\t test",len(pd.unique(test.track_id)) , 
          "\t val", len(pd.unique(val.track_id)))
    train_arr.append(len(pd.unique(train.track_id)))
    test_arr.append(len(pd.unique(test.track_id)))
    val_arr.append(len(pd.unique(val.track_id)))

DZ train 89 	 test 27 	 val 22
EG train 519 	 test 162 	 val 129
MA train 1 	 test 0 	 val 0
ZA train 1404 	 test 438 	 val 350
TN train 36 	 test 11 	 val 8
BH train 102 	 test 31 	 val 25
HK train 567 	 test 176 	 val 141
IN train 64 	 test 19 	 val 16
ID train 1108 	 test 345 	 val 276
IL train 1108 	 test 346 	 val 277
JP train 1556 	 test 486 	 val 389
JO train 23 	 test 7 	 val 5
KW train 28 	 test 8 	 val 7
LB train 10 	 test 2 	 val 2
MY train 370 	 test 115 	 val 92
OM train 1 	 test 0 	 val 0
PS train 8 	 test 2 	 val 2
PH train 785 	 test 245 	 val 196
QA train 2 	 test 0 	 val 0
SA train 4 	 test 1 	 val 1
SG train 408 	 test 127 	 val 101
TW train 1798 	 test 561 	 val 449
TH train 1136 	 test 355 	 val 284
AE train 91 	 test 28 	 val 22
VN train 1134 	 test 354 	 val 283
AD train 96 	 test 29 	 val 23
AT train 275 	 test 85 	 val 68
BE train 566 	 test 176 	 val 141
BG train 8 	 test 2 	 val 1
CY train 61 	 test 18 	 val 15
CZ train 58 	 test 18 	 val 14
DK train 2261 	 t

In [ ]:
Category = "Country"
Africa = ['DZ', 'EG', 'MA', 'ZA', 'TN']
Asia = ['BH', 'HK', 'IN', 'ID', 'IL', 'JP', 'JO', 'KW', 
        'LB', 'MY', 'OM', 'PS', 'PH', 'QA', 'SA', 'SG', 
        'TW', 'TH', 'AE', 'VN']
Europe = ['AD', 'AT', 'BE', 'BG', 'CY', 'CZ', 'DK', 'EE', 
          'FI', 'FR', 'DE', 'GR', 'HU', 'IS', 'IE', 'IT', 
          'LV', 'LI', 'LT', 'LU', 'MT', 'MC', 'NL', 'NO', 
          'PL', 'PT', 'RO', 'SK', 'ES', 'SE', 'CH', 'TR', 'GB']
NorthAmerica = ['CA', 'CR', 'DO', 'SV', 'GT', 'HN', 'MX', 'NI', 'PA', 'US']
SouthAmerica = ['AR', 'BO', 'BR', 'CL', 'CO', 'EC', 'PY', 'PE', 'UY']
Oceania = ['AU', 'NZ']


In [29]:
counts = pd.DataFrame([countriesOfInterest, train_arr, test_arr, val_arr]).T
counts.columns = ["Country", "train", "test", "val"]
counts["Continent"] = ""
counts.loc[counts.Country.isin(Africa), "Continent"] = "Africa"
counts.loc[counts.Country.isin(Asia), "Continent"] = "Asia"
counts.loc[counts.Country.isin(Europe), "Continent"] = "Europe"
counts.loc[counts.Country.isin(NorthAmerica), "Continent"] = "NorthAmerica"
counts.loc[counts.Country.isin(SouthAmerica), "Continent"] = "SouthAmerica"
counts.loc[counts.Country.isin(Oceania), "Continent"] = "Oceania"
counts

,Country,train,test,val,Continent
0,DZ,89,27,22,Africa
1,EG,519,162,129,Africa
2,MA,1,0,0,Africa
3,ZA,1404,438,350,Africa
4,TN,36,11,8,Africa
...,...,...,...,...,...
74,PY,9,2,2,SouthAmerica
75,PE,784,244,196,SouthAmerica
76,UY,19,5,4,SouthAmerica
77,AU,692,216,173,Oceania


In [42]:
cntGroup = counts.groupby(["Continent"]).apply(lambda x: x.sort_values(["train"], ascending = False)).reset_index(drop=True)
cntGroup.groupby(["Continent"]).head(2)

,Country,train,test,val,Continent
0,ZA,1404,438,350,Africa
1,EG,519,162,129,Africa
5,TW,1798,561,449,Asia
6,JP,1556,486,389,Asia
25,DK,2261,706,565,Europe
26,FI,1680,524,420,Europe
58,US,1547,483,386,NorthAmerica
59,CA,1010,315,252,NorthAmerica
68,AU,692,216,173,Oceania
69,NZ,434,135,108,Oceania
